In [1]:
import json

import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

from plot import vessel_planning

from vo_colos.utils.object_registry import SiteRegistry, VesselRegistry
from vo_colos.utils.object_registry import ClassInspect

In [5]:
config = {
    "tag": [
        "Cutter",
        "Barge",
        "example",
        "demo"
    ],
    "engine": "colos",
    "config": {
        "simulationProperties": {},
        "activities": [
            {
                "name": "SingeRunCycle1",
                "class": "WhileActivity",
                "children": [
                    "WhileFull1"
                ],
                "properties": {
                    "condition_event": [
                        {
                            "type": "container",
                            "concept": "to_site",
                            "state": "full"
                        }
                    ]
                }
            },
            {
                "name": "WhileFull1",
                "class": "SequenceActivity",
                "children": [
                    "sailing_empty1",
                    "loading1",
                    "sailing_full1",
                    "unloading1"
                ],
                "properties": {}
            },
            {
                "name": "sailing_empty1",
                "class": "MoveActivity",
                "properties": {
                    "mover": "barge_1",
                    "destination": "from_site"
                }
            },
            {
                "name": "loading1",
                "class": "ShiftAmountActivity",
                "properties": {
                    "processor": "cutter",
                    "origin": "from_site",
                    "destination": "barge_1",
                    "amount": 5,
                    "duration": 3600
                }
            },
            {
                "name": "sailing_full1",
                "class": "MoveActivity",
                "properties": {
                    "mover": "barge_1",
                    "destination": "to_site"
                }
            },
            {
                "name": "unloading1",
                "class": "ShiftAmountActivity",
                "properties": {
                    "processor": "barge_1",
                    "origin": "barge_1",
                    "destination": "to_site",
                    "amount": 5,
                    "duration": 3600
                }
            },
            {
                "name": "SingeRunCycle",
                "class": "WhileActivity",
                "children": [
                    "WhileFull"
                ],
                "properties": {
                    "condition_event": [
                        {
                            "type": "container",
                            "concept": "to_site",
                            "state": "full"
                        }
                    ]
                }
            },
            {
                "name": "WhileFull",
                "class": "SequenceActivity",
                "children": [
                    "sailing_empty",
                    "loading",
                    "sailing_full",
                    "unloading"
                ],
                "properties": {}
            },
            {
                "name": "sailing_empty",
                "class": "MoveActivity",
                "properties": {
                    "mover": "barge_2",
                    "destination": "from_site"
                }
            },
            {
                "name": "loading",
                "class": "ShiftAmountActivity",
                "properties": {
                    "processor": "cutter",
                    "origin": "from_site",
                    "destination": "barge_2",
                    "amount": 5,
                    "duration": 3600
                }
            },
            {
                "name": "sailing_full",
                "class": "MoveActivity",
                "properties": {
                    "mover": "barge_2",
                    "destination": "to_site"
                }
            },
            {
                "name": "unloading",
                "class": "ShiftAmountActivity",
                "properties": {
                    "processor": "barge_2",
                    "origin": "barge_2",
                    "destination": "to_site",
                    "amount": 5,
                    "duration": 3600
                }
            }
        ],
        "vessels": {
            "cutter": {
                "class": "TestCutter",
                "properties": {
                    "lat": 50,
                    "lon": 6,
                    "duplicates": 1,
                    "loading_rate": 1,
                    "nr_resources": 1,
                    "unloading_rate": 1,
                    "load_manoeuvring": 0,
                    "unload_manoeuvring": 0
                }
            },
            "barge_1": {
                "class": "TestBarge",
                "properties": {
                    "v": 1,
                    "lat": 50,
                    "lon": 6,
                    "level": 0,
                    "v_full": 8,
                    "v_empty": 10,
                    "capacity": 5,
                    "duplicates": 1,
                    "nr_resources": 1,
                    "unloading_rate": 1,
                    "unload_manoeuvring": 0
                }
            },
            "barge_2": {
                "class": "TestBarge",
                "properties": {
                    "v": 1,
                    "lat": 50,
                    "lon": 6,
                    "level": 0,
                    "v_full": 8,
                    "v_empty": 10,
                    "capacity": 5,
                    "duplicates": 1,
                    "nr_resources": 1,
                    "unloading_rate": 1,
                    "unload_manoeuvring": 0
                }
            }
        },
        "sites": {
            "from_site": {
                "class": "TestSite",
                "properties": {
                    "lat": 50,
                    "lon": 6,
                    "level": 55,
                    "capacity": 55,
                    "nr_resources": 1
                }
            },
            "to_site": {
                "class": "TestSite",
                "properties": {
                    "lat": 50,
                    "lon": 7,
                    "level": 0,
                    "capacity": 55,
                    "nr_resources": 1
                }
            }
        },
        "graph": {
            "nodes": [],
            "edges": []
        }
    }
}

In [6]:
"""File that defines the Coloses."""
import datetime
import json
import logging
import time
import uuid
from abc import ABC, abstractmethod
from typing import ClassVar, Dict

import openclsim.model as model
import shapely.geometry
import simpy

from vo_colos.utils.object_registry import SiteRegistry, VesselRegistry
from vo_colos.utils.serialise_graph import recreate_graph

logger = logging.getLogger(__name__)


class LogEncoder(json.JSONEncoder):
    """Class to catch all non-native types before json encoding."""

    def default(self, obj):
        if isinstance(obj, (datetime.datetime, datetime.date)):
            return obj.isoformat()
        if isinstance(obj, shapely.geometry.point.Point):
            return [obj.x, obj.y]
        return super().default(obj)


def make_jsonifyable(obj):
    """Guarantees object to be jsonifyable."""
    return json.loads(json.dumps(obj, cls=LogEncoder))


class BaseTask(ABC):
    """Abstract Base Class for the worker to execute the task."""

    # task registry
    registry: ClassVar[Dict[str, "BaseTask"]] = {}

    # task identification key
    key: str

    def __init_subclass__(cls, *args, **kwargs):
        """
        Register all classes that inherit from the `BaseTask` class.

        All child classes are stored in a class property `registry` under a
        identification key that is task-specific. The task-specific key must be stored
        in the class property `key` and must be a string. The task-specific key can be
        used as value for the `engine` property of a job.
        """
        if not cls.key:
            logger.warning("Task key is not defined.")
        elif not isinstance(cls.key, str):
            logger.warning(f"Task key is not a string: {cls.key}.")
        elif cls.key in cls.registry.items():
            logger.warning(f"Task with key '{cls.key}' already defined.'")
        else:
            cls.registry[str(cls.key)] = cls
        super().__init_subclass__(*args, **kwargs)

    def __init__(self, message):
        """Init of the task class."""
        self.message = message

    @abstractmethod
    def simulate(self):
        self.logs = []

    def get_logs(self):
        assert hasattr(self, "logs")
        assert isinstance(self.logs, list)

        database_logs = []

        for log in self.logs:
            assert isinstance(log, dict)
            assert isinstance(log.get("category"), str)
            assert isinstance(log.get("log"), dict)

            database_log = dict(
                id=uuid.uuid1(),
                job_id=int(self.message.id),
                project_id=self.message.task["project_id"],
                category=log["category"],
                log=make_jsonifyable(log["log"]),
            )

            database_logs.append(database_log)

        return database_logs


class SimulationFactory(BaseTask):
    """Factory for the Colos Simulations."""

    key = "colos"

    def build_activity(self, act, postpone_start=False):
        objects = list(self.locations)
        objects.extend(self.vessels)

        keep_resources = act["properties"].get("keep_resources")
        if keep_resources:
            keep_resources = [self.get_from_list(k, objects) for k in keep_resources]
        else:
            keep_resources = []

        if act["class"] == "ShiftAmountActivity":
            processor = self.get_from_list(act["properties"]["processor"], objects)
            origin = self.get_from_list(act["properties"]["origin"], objects)
            destination = self.get_from_list(act["properties"]["destination"], objects)

            obj = model.ShiftAmountActivity(
                name=act["name"],
                env=self.env,
                registry=self.registry,
                requested_resources=self.requested_resources,
                processor=processor,
                origin=origin,
                destination=destination,
                amount=act["properties"]["amount"],
                duration=act["properties"]["duration"],
                postpone_start=postpone_start,
                keep_resources=keep_resources,
            )
            return obj

        if act["class"] == "MoveActivity":
            mover = self.get_from_list(act["properties"]["mover"], objects)
            destination = self.get_from_list(act["properties"]["destination"], objects)

            obj = model.MoveActivity(
                name=act["name"],
                env=self.env,
                registry=self.registry,
                requested_resources=self.requested_resources,
                mover=mover,
                destination=destination,
                postpone_start=postpone_start,
                keep_resources=keep_resources,
            )
            return obj

        if act["class"] == "WhileActivity":
            sub = [a for a in self.activities if a.name in act["children"]]
            assert len(sub) == 1

            condition_event = act["properties"]["condition_event"]
            for event in range(len(condition_event)):
                concept = condition_event[event].get("concept")
                concept = self.get_from_list(concept, objects)
                condition_event[event]["concept"] = concept

            obj = model.WhileActivity(
                name=act["name"],
                env=self.env,
                registry=self.registry,
                requested_resources=self.requested_resources,
                postpone_start=postpone_start,
                condition_event=act["properties"]["condition_event"],
                sub_process=sub[0],
                keep_resources=keep_resources,
            )
            return obj

        if act["class"] == "SequenceActivity":
            sub = [a for a in self.activities if a.name in act["children"]]
            obj = model.SequentialActivity(
                name=act["name"],
                env=self.env,
                registry=self.registry,
                requested_resources=self.requested_resources,
                postpone_start=postpone_start,
                sub_processes=sub,
                keep_resources=keep_resources,
            )
            return obj

    def environment_factory(self):
        """Return the OpenCLSim environment."""
        simulation_start = self.config["simulationProperties"].get("simulation_start")
        if not simulation_start:
            simulation_start = datetime.datetime.now()
            simulation_start = time.mktime(simulation_start.timetuple())

        self.env = simpy.Environment(initial_time=simulation_start)
        self.env.epoch = simulation_start

        FG = recreate_graph(self.config["graph"])

        self.env.FG = FG

    def site_factory(self):
        self.locations = []
        for site_name in self.config["sites"]:
            class_name = self.config["sites"][site_name]["class"]
            site_object = SiteRegistry.registry.get(class_name)

            properties = self.config["sites"][site_name]["properties"]

            data = {"env": self.env, "name": site_name, "ID": site_name, **properties}
            self.locations.append(site_object(**data))

    def vessel_factory(self):
        self.vessels = []
        for vessel_name in self.config["vessels"]:
            vessel_class = self.config["vessels"][vessel_name]["class"]
            vessel_object = VesselRegistry.registry.get(vessel_class)
            properties = self.config["vessels"][vessel_name]["properties"]
            data = {"env": self.env, "name": vessel_name, **properties}
            self.vessels.append(vessel_object(**data))

    def get_from_list(self, name, array):
        obj = [a for a in array if a.name == name]
        assert len(obj) == 1
        return obj[0]

    def activity_factory(self):
        self.activities = []

        basic_activites = ["MoveActivity", "ShiftAmountActivity", "BasicActivity"]
        self.registry = {}
        self.requested_resources = {}

        total_children = []
        for child in [act.get("children", []) for act in self.config["activities"]]:
            total_children.extend(child)

        for act in self.config["activities"]:
            if act["class"] in basic_activites:
                self.activities.append(
                    self.build_activity(
                        act, postpone_start=act["name"] in total_children
                    )
                )

        i = 0
        names = set([a["name"] for a in self.config["activities"]])
        while set([a.name for a in self.activities]) != names:
            acts = [
                a
                for a in self.config["activities"]
                if a["name"] not in [a.name for a in self.activities]
            ]
            act = acts[i]
            activity_names = [a.name for a in self.activities]
            dependencies = [c in activity_names for c in act.get("children", [])]

            if act["name"] not in activity_names and False not in dependencies:
                self.activities.append(
                    self.build_activity(
                        act, postpone_start=act["name"] in total_children
                    )
                )
                i = 0
            else:
                i = i + 1
                assert i < len(acts)

    def simulate(self):
        self.config = self.message.task["config"]

        self.environment_factory()
        self.site_factory()
        self.vessel_factory()
        self.activity_factory()

        self.env.run()

        self.logs = []

        for log in self.activities:
            self.logs.append({"category": "Activity log", "log": log.log})
        for log in self.vessels:
            self.logs.append({"category": "Vessel log", "log": log.log})
        for log in self.locations:
            self.logs.append({"category": "Site log", "log": log.log})


In [7]:
class message:
    id = 1
    task = {
        "id": 1,
        "engine": "colos",
        "project_id": "1234567890",
        "tag": "testsimulation",
        "config": config["config"],
    }
    status = "new"

sim = SimulationFactory(message)

In [8]:
sim.simulate()

init
level: 55
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
while Activity keep_resources []
get_full_event : default
start get_available
start event instance None
while Activity keep_resources []
get_full_event : default
start get_available
start event instance None
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{'default': {55: <Event() object at 0x11c6e398c88>}}
amount :55
put_callback - id_ default
{}
put_callback - id_ default
{}
Mover_move after mover resource request
Mover_move after mover resource request
Mover_move after move
Activity end(sailing_empty1)
Mover_move after move
Activity end(sailing_empty)
keep_resources []
SHift amount Activity keep_resources []
start e

transfer default to to_site
processor process without rate
origin store before get: [{'id': 'default', 'level': 5, 'capacity': 5}]
start get 5
store_status {'id': 'default', 'level': 5, 'capacity': 5}
end get 5
get_callback - id_ <StorePut() object at 0x11c6e3b9988>
start get_callback
{}
origin store after get: [{'id': 'default', 'level': 0, 'capacity': 5}]
after get
after check_down time
get_callback - id_ <StorePut() object at 0x11c6e329cc8>
start get_callback
{}
origin store after get: [{'id': 'default', 'level': 0, 'capacity': 5}]
after get
after check_down time
destination store before put: [{'id': 'default', 'level': 30, 'capacity': 55}]
<FilterStoreGet() object at 0x11c64a62fc8>
destination store before put: [{'id': 'default', 'level': 35, 'capacity': 55}]
<FilterStoreGet() object at 0x11c6e329cc8>
put_callback - id_ default
{'default': {55: <Event() object at 0x11c6e398c88>}}
amount :55
destination store after put: [{'id': 'default', 'level': 40, 'capacity': 55}]
after put
proc

In [9]:
objects = list(sim.vessels)
activities = []
for obj in objects:
    print(set(obj.log["Message"]))
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

            
C = np.linspace(0,255, len(activities))
colors = {}

for i in range(len(activities)):
    colors[i] = f'rgb({int(C[i]/2)},{int(C[len(C) - 1 - i]/1.5)},{int(C[i])})'

vessel_planning(objects, activities, colors)

{'transfer default to barge_2', 'transfer default to barge_1'}
{'transfer default to to_site', 'sailing empty', 'transfer default to barge_1', 'sailing filled'}
{'transfer default to barge_2', 'sailing empty', 'transfer default to to_site', 'sailing filled'}
